In [114]:
import os
from dotenv import load_dotenv
load_dotenv()
import faiss
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
import pandas as pd
import sqlite3
from langchain.prompts import ChatPromptTemplate
from llama_index.llms.groq import Groq
import fitz

In [115]:
groq_api_key = os.getenv("GROQ_API_KEY")

# RAG LLM Query Retrival

In [116]:
SQL_DB_PATH = "../database/csv_database.db"
FAISS_INDEX_PATH = "../database/faiss_index"

In [117]:
# embedding function
def get_embeddings():
    embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
    return embeddings

In [118]:
# Prompt Template
PROMPT_TEMPLATE ="""
    Jawab pertanyaan berdasarkan konteks dari PDF berikut: 

    {context}

    ---

    **Pertanyaan:**
    {question}

    ---
    Hanya lanjutkan ke bagian di bawah ini jika benar-benar tidak ada informasi spesifik dalam konteks.
    Jika pertanyaan yang diberikan terkait data riwayat penggunaan dan tidak ada informasi spesifik dalam konteks, jawablah dengan **SQL query** yang sesuai dengan contoh tabel berikut:

    **Nama Tabel:** `synthetic_data`

    **Contoh Data dalam Tabel (Format JSON, hanya contoh, tidak digunakan langsung dalam query):**
    ```json
    [
        {{"interaction_id":1,"user_id":447,"timestamp":"2024-03-12 17:56:48","device_type":"Smart Speaker","command_category":"Information","command_text":"Apa rekomendasi restoran di sekitar sini?","ai_response":"Akses ditolak","response_time_ms":310,"ai_confidence_score":67.97,"user_satisfaction":4,"status":"Error","error_code":"ERR403"}},
        {{"interaction_id":2,"user_id":469,"timestamp":"2024-12-01 06:40:13","device_type":"Smartphone","command_category":"Productivity","command_text":"Bagikan agenda meeting dengan anggota tim","ai_response":"Tugas telah disimpan","response_time_ms":315,"ai_confidence_score":70.04,"user_satisfaction":5,"status":"Success","error_code":null}}
    ]
    ```
    Hanya jawab query SQL dalam bentuk string, tanpa memberi jawaban yang lain.

    ---
    Hanya jawab query SQL, jika tidak ada data yang sesuai dalam konteks.
"""

PROMPT_DB = """
    Jawab pertanyaan berdasarkan hasil query SQL yang diberikan di bawah ini:
    {sql_query_result}

    ---
    Jawab pertanyaan berikut:
    {question}

    Jawab tanpa menyebutkan penggunaan query SQL, tetapi parafrase pertanyaan untuk memberi jawaban yang natural.
"""


In [119]:
def query_retrieval_rag(question: str):
    # Load LLM model
    llm = Groq(model="llama-3.1-8b-instant", api_key=groq_api_key)

    # Load embeddings & vector store
    embeddings = get_embeddings()
    vector_store = FAISS.load_local(FAISS_INDEX_PATH, embeddings, allow_dangerous_deserialization=True)

    # Search vector store
    results = vector_store.similarity_search(question, k=5)
    if not results:
        return "Maaf, saya tidak menemukan informasi yang relevan dalam dokumen."

    context_text = '\n\n---\n\n'.join(doc.page_content for doc in results)

    # Prepare prompt with clear formatting
    prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
    prompt = prompt_template.format(context=context_text, question=question)

    # Generate response using LLM
    response = llm.complete(prompt)
    response = str(response)

    # Validate if response contains an SQL query
    if "synthetic_data" in response:  
        # Take SQL query from response
        response = response.split("```sql")[1].strip()
        response = response.split("```")[0].strip()

        try:
            # Connect to SQLite database
            conn = sqlite3.connect(SQL_DB_PATH)

            # Execute SQL query safely
            sql_query_result = pd.read_sql_query(response, conn)
            conn.close()

            # If DataFrame is empty, return error message
            if sql_query_result.empty:
                return "Query berhasil dijalankan, tetapi tidak ada hasil yang ditemukan."

            # Convert DataFrame to JSON format string
            query_result_str = sql_query_result.to_json(orient="records")

            # Prepare second-stage prompt (SQL result → natural language)
            prompt_db = ChatPromptTemplate.from_template(PROMPT_DB)
            final_prompt = prompt_db.format(sql_query_result=query_result_str, question=question)

            # Generate final response
            response = llm.complete (final_prompt)

            return response
        
        except sqlite3.Error as e:
            return f"Terjadi kesalahan SQL: {e}"
    
    return response

In [120]:
# prompt
question = 'Berapa rata-rata kepuasan pengguna AI Assistant X-3000 sebelumnya?'
response = query_retrieval_rag(question)
print(response)

Rata-rata kepuasan pengguna AI Assistant X-3000 sebelumnya adalah 3,44.


# Baseline Query Retrival

In [121]:
PDF_PATH = "../files/synthetic_data.pdf"
CSV_PATH = "../files/synthetic_data.csv"

In [122]:
PROMPT_TEMPLATE_BASELINE = """
Pertanyaan:
{question}

Jawablah dengan informasi yang paling relevan dari dokumen di bawah ini.

---

Jawab pertanyaan berdasarkan informasi dalam dokumen dan sample data berikut:

Dokumen:
{full_document}

Sample Data:
{sample_data}

---

Jangan berikan code, hanya berikan jawaban dengan pernyataan natural.

"""

In [123]:
def extract_full_text_from_pdf(pdf_path):
    """Extract all text from a PDF document to use as baseline LLM input."""
    doc = fitz.open(pdf_path)
    full_text = "\n".join([page.get_text("text") for page in doc])
    full_text = ' '.join(full_text.split())
    # cut off the text to 10000 karakter
    full_text = full_text[:10000]
    return full_text

In [124]:
def extract_full_csv_data(csv_path):
    """Extract all text from a CSV document to use as baseline LLM input."""
    df = pd.read_csv(csv_path)
    # take the first 20 rows
    df = df.head(20)
    full_text = df.to_string()
    return full_text

In [125]:
def query_retrieval_baseline(question: str):
    # Load LLM model
    llm = Groq(model="llama-3.1-8b-instant", api_key=groq_api_key)

    # Extract full document text
    full_document = extract_full_text_from_pdf(PDF_PATH)
    sample_data = extract_full_csv_data(CSV_PATH)

    # Prepare prompt with clear formatting
    prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE_BASELINE)
    prompt = prompt_template.format(question=question, full_document=full_document, sample_data=sample_data)

    # Generate response using LLM
    response = llm.complete(prompt)
    response = str(response)
    
    return response

In [126]:
# prompt
question = 'Berapa rata-rata kepuasan pengguna AI Assistant X-3000 sebelumnya?'
response = query_retrieval_baseline(question)
print(response)

Rata-rata kepuasan pengguna AI Assistant X-3000 sebelumnya adalah 4,2 dari 5.


# RAG LLM Evaluation

In [129]:
# Question and Expected Response
validation_data = [
    {
        "question": "Apa fitur utama AI Assistant X-3000??",
    },
    {
        "question": "Bagaimana cara troubleshooting jika AI tidak merespons perintah suara?",
    },
    {
        "question": "Kategori perintah apa yang sering digunakan pengguna?",
    },
    {
        "question": "Berapa persentase error yang dialami pengguna sebelumnya?",
    },
    {
        "question": "Berapa rata-rata kepuasan pengguna AI Assistant X-3000 sebelumnya?",
    },
    {
        "question": "Jelaskan cara menghubungkan AI Assistant X-3000 ke Wi-Fi.",
    },
    {
        "question": "Apa teknologi yang digunakan untuk pengenalan wajah AI Assistant X-3000?",
    },
    {
        "question": "Apa langkah pemeliharaan rutin AI Assistant X-3000?",
    },
    {
        "question": "Jelaskan perbedaan AI Assistant X-3000 dengan model sebelumnya.",
    },
    {
        "question": "Apa manfaat AI Assistant X-3000 dalam smart home?",
    }
]

In [130]:
for i, data in enumerate(validation_data):
    question = data["question"]

    # Retrieve response from model
    rag_response = query_retrieval_rag(question)
    baseline_response = query_retrieval_baseline(question)

    # Print evaluation prompt
    print(f"QUESTION {i+1}")
    print(f"Question: {question}")
    print(f"RAG Response: {rag_response}")
    print(f"Baseline Response: {baseline_response}")
    print("\n")

QUESTION 1
Question: Apa fitur utama AI Assistant X-3000??
RAG Response: Fitur utama AI Assistant X-3000 adalah kemampuannya dalam memproses perintah suara dan memberikan respons secara visual maupun auditori, serta kemampuan navigasi otomatis dan penghindaran hambatan dengan menggunakan sensor LiDAR, kamera RGB, dan sensor ultrasonik.
Baseline Response: Fitur utama AI Assistant X-3000 adalah:

1. Interaksi Suara dan Visual: AI Assistant X-3000 dapat memproses perintah suara dan memberikan respons secara visual maupun auditori.
2. Navigasi Otomatis dan Penghindaran Hambatan: Robot ini dapat bergerak secara mandiri dan menghindari rintangan dengan akurasi tinggi.
3. Asisten AI yang Dapat Menyesuaikan Diri: AI Assistant X-3000 dapat beradaptasi dengan kebiasaan dan preferensi pengguna.
4. Integrasi dengan Perangkat Pintar (IoT): Robot ini dapat berkomunikasi dengan berbagai perangkat pintar, seperti lampu otomatis, pendingin udara, sistem keamanan rumah, dan speaker pintar.
5. Pemrosesan

In [131]:
# Define benchmark questions
benchmark_questions = [
    "Apa fitur utama AI Assistant X-3000?",
    "Bagaimana cara troubleshooting jika AI tidak merespons perintah suara?",
    "Kategori perintah apa yang sering digunakan pengguna?",
    "Berapa persentase error yang dialami pengguna sebelumnya?",
    "Berapa rata-rata kepuasan pengguna AI Assistant X-3000 sebelumnya?",
    "Jelaskan cara menghubungkan AI Assistant X-3000 ke Wi-Fi.",
    "Apa teknologi yang digunakan untuk pengenalan wajah AI Assistant X-3000?",
    "Apa langkah pemeliharaan rutin AI Assistant X-3000?",
    "Jelaskan perbedaan AI Assistant X-3000 dengan model sebelumnya.",
    "Apa manfaat AI Assistant X-3000 dalam smart home?"
]

# Define scores (manually assigned based on evaluation)
data = {
    "Question": benchmark_questions,
    "Baseline_Accuracy": [5, 3, 3, 2, 3, 4, 3, 4, 5, 3],
    "Baseline_Relevance": [5, 2, 3, 1, 2, 4, 3, 4, 5, 3],
    "Baseline_Completeness": [5, 3, 4, 2, 3, 4, 3, 4, 5, 3],
    "Baseline_Conciseness": [4, 5, 4, 4, 5, 5, 5, 5, 5, 5],
    "RAG_Accuracy": [4, 5, 4, 5, 5, 5, 5, 5, 5, 5],
    "RAG_Relevance": [4, 5, 5, 5, 5, 5, 5, 5, 5, 5],
    "RAG_Completeness": [3, 5, 3, 5, 5, 5, 5, 5, 5, 5],
    "RAG_Conciseness": [5, 5, 5, 5, 5, 5, 5, 5, 5, 5]
}

# Convert to DataFrame
df = pd.DataFrame(data)

# Define function to calculate final scores
def evaluate_answer(accuracy, relevance, completeness, conciseness):
    return round((accuracy + relevance + completeness + conciseness) / 4, 2)

# Calculate final scores for Baseline and RAG
df["Baseline_Score"] = df.apply(lambda row: evaluate_answer(row["Baseline_Accuracy"], row["Baseline_Relevance"], row["Baseline_Completeness"], row["Baseline_Conciseness"]), axis=1)
df["RAG_Score"] = df.apply(lambda row: evaluate_answer(row["RAG_Accuracy"], row["RAG_Relevance"], row["RAG_Completeness"], row["RAG_Conciseness"]), axis=1)

# Display results
df


,Question,Baseline_Accuracy,Baseline_Relevance,Baseline_Completeness,Baseline_Conciseness,RAG_Accuracy,RAG_Relevance,RAG_Completeness,RAG_Conciseness,Baseline_Score,RAG_Score
0,Apa fitur utama AI Assistant X-3000?,5,5,5,4,4,4,3,5,4.75,4.00
1,Bagaimana cara troubleshooting jika AI tidak m...,3,2,3,5,5,5,5,5,3.25,5.00
2,Kategori perintah apa yang sering digunakan pe...,3,3,4,4,4,5,3,5,3.50,4.25
3,Berapa persentase error yang dialami pengguna ...,2,1,2,4,5,5,5,5,2.25,5.00
4,Berapa rata-rata kepuasan pengguna AI Assistan...,3,2,3,5,5,5,5,5,3.25,5.00
5,Jelaskan cara menghubungkan AI Assistant X-300...,4,4,4,5,5,5,5,5,4.25,5.00
6,Apa teknologi yang digunakan untuk pengenalan ...,3,3,3,5,5,5,5,5,3.50,5.00
7,Apa langkah pemeliharaan rutin AI Assistant X-...,4,4,4,5,5,5,5,5,4.25,5.00
8,Jelaskan perbedaan AI Assistant X-3000 dengan ...,5,5,5,5,5,5,5,5,5.00,5.00
9,Apa manfaat AI Assistant X-3000 dalam smart home?,3,3,3,5,5,5,5,5,3.50,5.00


In [132]:
# Print overall averages
baseline_avg = round(df["Baseline_Score"].mean(), 2)
rag_avg = round(df["RAG_Score"].mean(), 2)

print(f"\n🔹 Baseline LLM Average Score: {baseline_avg}/5.0")
print(f"🔹 RAG System Average Score: {rag_avg}/5.0")


🔹 Baseline LLM Average Score: 3.75/5.0
🔹 RAG System Average Score: 4.82/5.0
